In [ ]:
# https://youtu.be/9GzfUzJeyi0

"""
@author: Sreenivas Bhattiprolu
"""

import numpy as np 
import matplotlib.pyplot as plt
import glob
import cv2
import torch
import torch.backends.cudnn as cudnn

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.layers import Dense, Flatten, Conv2D, MaxPooling2D, BatchNormalization


#from keras.layers.normalization import BatchNormalization
import os
import seaborn as sns
from tensorflow.keras.applications.vgg16 import VGG16


total_classes=0  #to change when mee lay want to add more cows
#images_per_validation=5


#print(os.listdir("../Gold"))

SIZE = 112

import csv

#data = pd.read_csv('D:\\LamenessData\\September_6\\evening.csv',index_col ='Local_ID')

#COW_MAPPER = [list(row) for row in data.values]

x_train = []
train_labels = [] 
directory_path = "D:\\LamenessData\\yolov8_features\\honkawa_yolov8_dataset291\\TrainingImages\\"
label = "CATTLE"

t =  os.path.join(directory_path, "*.jpg")
print(t)
for img_path in glob.glob(os.path.join(directory_path, "*.jpg")):
    #print(img_path)
    total_classes+=1
    img = cv2.imread(img_path, cv2.IMREAD_COLOR)       
        
    img = cv2.resize(img, (SIZE, SIZE))
    img = cv2.cvtColor(img, cv2.COLOR_RGB2BGR)
    x_train.append(img)
    train_labels.append(label)
        
x_train = np.array(x_train)
train_labels = np.array(train_labels)

print(total_classes)




In [ ]:

# test
count = 0
y_train = []
test_labels = [] 
images_per_validation =[]
directory_path = "D:\\LamenessData\\yolov8_features\\honkawa_yolov8_dataset291\\\ValidationImages\\"  #to change when baby add new images
for img_path in glob.glob(os.path.join(directory_path, "*.jpg")):
    #print(img_path)
    total_classes+=1
    img = cv2.imread(img_path, cv2.IMREAD_COLOR)       
        
    img = cv2.resize(img, (SIZE, SIZE))
    img = cv2.cvtColor(img, cv2.COLOR_RGB2BGR)

    y_train.append(img)
    test_labels.append("Validation")
    count+=1
    images_per_validation.append(count)
        
y_train = np.array(y_train)
test_labels = np.array(test_labels)
#Encode labels from text to integers.
from sklearn import preprocessing
le = preprocessing.LabelEncoder()
le.fit(test_labels)
test_labels_encoded = le.transform(test_labels)
le.fit(train_labels)
train_labels_encoded = le.transform(train_labels)
print("Train lables encoded is here")
#print(train_labels_encoded)
#Split data into test and train datasets (already split but assigning to meaningful convention)




In [36]:


# Sample dataset (you should replace this with your own dataset)
#(x_train, y_train), (x_test, y_test) = keras.datasets.mnist.load_data()
x_train, y_train, x_test, y_test = x_train, train_labels_encoded, y_train, test_labels_encoded
x_train = x_train / 255.0
x_test = x_test / 255.0

# Define a Siamese network architecture
def create_siamese_network(input_shape):
    input = keras.layers.Input(shape=input_shape)
    x = keras.layers.Flatten()(input)
    x = keras.layers.Dense(128, activation='relu')(x)
    x = keras.layers.Dropout(0.5)(x)
    output = keras.layers.Dense(64)(x)
    return keras.models.Model(input, output)

input_shape = x_train.shape[1:]
print(input_shape)
# Create two identical siamese subnetworks
base_network = create_siamese_network(input_shape)

input_anchor = keras.layers.Input(shape=input_shape)
input_positive = keras.layers.Input(shape=input_shape)
input_negative = keras.layers.Input(shape=input_shape)

output_anchor = base_network(input_anchor)
output_positive = base_network(input_positive)
output_negative = base_network(input_negative)

# Define the triplet loss
def triplet_loss(margin=0.2):
    def loss(y_true, y_pred):
        anchor, positive, negative = y_pred[0], y_pred[1], y_pred[2]
        distance_positive = tf.reduce_sum(tf.square(anchor - positive), axis=-1)
        distance_negative = tf.reduce_sum(tf.square(anchor - negative), axis=-1)
        loss_value = tf.maximum(0.0, distance_positive - distance_negative + margin)
        return tf.reduce_mean(loss_value)
    return loss

# Create the Siamese model
siamese_model = keras.models.Model(inputs=[input_anchor, input_positive, input_negative], outputs=[output_anchor, output_positive, output_negative])

# Compile the model with the triplet loss
siamese_model.compile(loss=triplet_loss(), optimizer=keras.optimizers.Adam(learning_rate=0.001))

# Train the model using triplets
# You need to provide your own triplet data
# Here, we'll use dummy data
n_triplets = 10000
anchor, positive, negative = x_train[:n_triplets], x_train[n_triplets:2*n_triplets], x_train[2*n_triplets:3*n_triplets]
#print(anchor)
siamese_model.fit([anchor, positive, negative], y=x_test[:n_triplets], epochs=20, batch_size=8)

# After training, you can use the base_network to extract embeddings for inference.
print(done)

(112, 112, 3)
Epoch 1/20
1250/1250 [==============================] - 38s 30ms/step - loss: 0.6000 - model_14_loss: 0.2000 - model_14_1_loss: 0.2000 - model_14_2_loss: 0.2000
Epoch 2/20
1250/1250 [==============================] - 36s 29ms/step - loss: 0.6000 - model_14_loss: 0.2000 - model_14_1_loss: 0.2000 - model_14_2_loss: 0.2000
Epoch 3/20
1250/1250 [==============================] - 38s 30ms/step - loss: 0.6000 - model_14_loss: 0.2000 - model_14_1_loss: 0.2000 - model_14_2_loss: 0.2000
Epoch 4/20
1250/1250 [==============================] - 38s 31ms/step - loss: 0.6000 - model_14_loss: 0.2000 - model_14_1_loss: 0.2000 - model_14_2_loss: 0.2000
Epoch 5/20
1250/1250 [==============================] - 38s 31ms/step - loss: 0.6000 - model_14_loss: 0.2000 - model_14_1_loss: 0.2000 - model_14_2_loss: 0.2000
Epoch 6/20
1250/1250 [==============================] - 38s 31ms/step - loss: 0.6000 - model_14_loss: 0.2000 - model_14_1_loss: 0.2000 - model_14_2_loss: 0.2000
Epoch 7/20
1250/1250

NameError: name 'done' is not defined